In [1]:
# This code block stores file paths to variables to make the code neat
# The stored variables are called in the read_excel function and stored as dataframes

import pandas as pd
import numpy as np
# Store excel file locations to variables

Com_2012 = r"C:\Users\warra\Desktop\Freelance\data\data\CommunityMerge\1. Merging by Parts\10. S6 Health Facility Type and health professionals\2012_s3_community.csv"
Com_2013 = r"C:\Users\warra\Desktop\Freelance\data\data\CommunityMerge\1. Merging by Parts\10. S6 Health Facility Type and health professionals\2013_s5_focus.csv"
Com_2014 = r"C:\Users\warra\Desktop\Freelance\data\data\CommunityMerge\1. Merging by Parts\10. S6 Health Facility Type and health professionals\2014_s7_community.csv"

# Read excel files 
df_2012 = pd.read_csv(Com_2012)
df_2013 = pd.read_csv(Com_2013)
df_2014 = pd.read_csv(Com_2014)



In [2]:
#This code block will be used to standardize column names across the years to avoid discrepancies during the merging process.

# Rename columns in df to df_2012 column names
df_2012.rename(columns={
    'health_id': 's7_health_id',
    'S3Q1': 's7_q1',
    'S3Q2': 's7_q2',
    'S3Q7a': 's7_q5',
    'S3Q8a': 's7_q6',
    'S3Q9a': 's7_q7',
    'S3Q3': 'HF_SFA_y',
    'S3Q4': 'HF_CA_DHCW',
    'S3Q5': 'HF_CA_MCHB',
    'S3Q6': 'HF_Quality',
    'S3Q7b': 'HF_IVPM_Name',
    'S3Q8b': 'HD_FPCampM_Name',
    'S3Q9b': 'HF_HEM_Name',
    'round': 'Survey_Round',
    'community': 'Community',
    'fid': 'FID',
    'PROVINCE_ID': 'P_ID',
    'DISTRICT_ID': 'D_ID',
    'TEHSIL_ID': 'T_ID',
    'MAUZA_ID': 'M_ID'
}, inplace=True)

# Rename columns in df to df_2013 column names
df_2013.rename(columns={
    'facility_id': 's7_health_id',
    'health_type': 's7_health_type',
    's5_q1': 's7_q1',
    's5_q2': 's7_q2',
    's5_q3': 's7_q3'
}, inplace=True)

# df_2014 doesn't need renaming as it is the reference

In [3]:
# Define the column mappings based on the provided positions
# Make dictionaries for each year with updated names
# replace with 'None' where there are no columns
#Here are the updated mapping lists for the given datasets:


mapping_2012 = [
    'cid',
    's7_health_id',
    None,
    's7_q1',
    's7_q2',
    None,
    None,
    's7_q5',
    's7_q6',
    's7_q7',
    'HF_SFA_y',
    'HF_CA_DHCW',
    'HF_CA_MCHB',
    'HF_Quality',
    'HF_IVPM_Name',
    'HD_FPCampM_Name',
    'HF_HEM_Name',
    'Survey_Round',
    'Community',
    'FID',
    'P_ID',
    'D_ID',
    'T_ID',
    'UC_ID',
    'M_ID'
]

mapping_2013 = [
    'cid',
    's7_health_id',
    's7_health_type',
    's7_q1',
    's7_q2',
    's7_q3',
    None, None, None, None, None, None, None, None, None, None, None, None,
    None, None, None, None, None,None, None
]


mapping_2014 = [
    'cid',
    's7_health_id',
    's7_health_type',
    's7_q1',
    's7_q2',
    's7_q3', 's7_q4', 's7_q5', 's7_q6','s7_q7',
    None, None, None, None, None, None, None, None, None, None, None, None,
    None, None, None
]





In [4]:
# Create a list of all possible columns in the correct order
all_columns = []

for col in mapping_2012:
    if col and col not in all_columns:
        all_columns.append(col)   
for col in mapping_2013:
    if col and col not in all_columns:
        all_columns.append(col)
for col in mapping_2014:
    if col and col not in all_columns:
        all_columns.append(col)



In [5]:
def standardize_and_merge(dfs, mappings, all_columns):
    merged_data = {col: [] for col in all_columns}

    for df, mapping in zip(dfs, mappings):
        print(f"Processing DataFrame with columns: {df.columns.tolist()}")
        for i, col in enumerate(mapping):
            if col:
                ref_col = col.strip()  # Remove leading/trailing whitespace
                if ref_col not in merged_data:
                    merged_data[ref_col] = []
                if ref_col in df.columns:
                    print(f"Appending data for column {ref_col}")
                    if isinstance(df[ref_col], pd.Series):
                        merged_data[ref_col].extend(df[ref_col].tolist())
                    elif isinstance(df[ref_col], pd.DataFrame):
                        print(f"Column {ref_col} is duplicated in DataFrame. Appending data for each duplicate.")
                        for _, series in df[ref_col].items():
                            merged_data[ref_col].extend(series.tolist())
                else:
                    print(f"Column {ref_col} not found in DataFrame. Adding NaNs.")
                    merged_data[ref_col].extend([np.nan] * len(df))
    
    max_len = max(len(v) for v in merged_data.values())
    for key in merged_data:
        col_len = len(merged_data[key])
        if col_len < max_len:
            merged_data[key].extend([np.nan] * (max_len - col_len))

    merged_df = pd.DataFrame.from_dict(merged_data)
    return merged_df


In [6]:
# Usage with dataframes and mappings
dfs = [df_2012, df_2013, df_2014]
mappings = [mapping_2012, mapping_2013, mapping_2014]

merged_df = standardize_and_merge(dfs, mappings, all_columns)

Processing DataFrame with columns: ['Unnamed: 0', 'Survey_Round', 'cid', 'Community', 's7_health_id', 's7_q1', 's7_q2', 'HF_SFA_y', 'HF_CA_DHCW', 'HF_CA_MCHB', 'HF_Quality', 'FID', 'P_ID', 'D_ID', 'T_ID', 'UC_ID', 'M_ID', 's7_q5', 'HF_IVPM_Name', 's7_q6', 'HD_FPCampM_Name', 's7_q7', 'HF_HEM_Name']
Appending data for column cid
Appending data for column s7_health_id
Appending data for column s7_q1
Appending data for column s7_q2
Appending data for column s7_q5
Appending data for column s7_q6
Appending data for column s7_q7
Appending data for column HF_SFA_y
Appending data for column HF_CA_DHCW
Appending data for column HF_CA_MCHB
Appending data for column HF_Quality
Appending data for column HF_IVPM_Name
Appending data for column HD_FPCampM_Name
Appending data for column HF_HEM_Name
Appending data for column Survey_Round
Appending data for column Community
Appending data for column FID
Appending data for column P_ID
Appending data for column D_ID
Appending data for column T_ID
Appending

In [7]:
# Rename columns for the merged file (if needed)
rename_mapping = {
    'cid': 'CID',
    's7_health_id': 'HF_ID',
    's7_health_type': 'HF_Type',
    's7_q1': 'HF_AddM',
    's7_q2': 'HF_AddM_DistKM',
    's7_q3': 'HF_AddM_TimeMins',
    's7_q4': 'HF_T',
    's7_q5': 'HF_IVPM',
    's7_q6': 'HD_FPCampM',
    's7_q7': 'HF_HEM'
}


merged_df.rename(columns=rename_mapping, inplace=True)

# Drop redundant columns
df_2014.drop(df_2014.columns[df_2014.columns.str.contains('Unnamed', case=True)], axis=1, inplace=True)
df_2014.drop(df_2014.columns[df_2014.columns.str.contains(' ', case=False)], axis=1, inplace=True)

# Save the merged dataframe to a CSV file
merged_df.to_csv('8. merged_S6_Health Facility Type and health professionals.csv', index=False)

